In [ ]:
"""
Unabashedly following Jake van der Plas' excellent overview paper:
https://iopscience.iop.org/article/10.3847/1538-4365/aab766

Lecture 1 Outline:

0. Motivation: what is real data like? nonuniform, different sized error bars.
   Why LS? It's the most common, and its generally fast. Major classes of periodicity 
   detection: phase folding, fourier-based, least-squares model-fitting 
1. Brief restatement of Fourier stuff from Bryan's lesson
2. Effect of (uniform) sampling (3.2, fig 7)
3. Effect of nonuniform sampling (4, figs 9 & 10)
4. Effect of semistructured observing windows (4.2, figs 14 & 15)
5. Definitions: beating, aliasing, window function

Problem ideas:
- given a window function and a true signal, draw the LS periodogram
- identify beat frequencies and alias frequencies in LS of real data
- decide a follow-up strategy based on a window function & a LS periodogram
- (more)

Lecture 2 Outline:

1. How LS relates to classical periodogram relates to PSD (they are estimators of PSD)
2. Sketch derivations of LS: smart choice of coefficients (Scargle) + least squares (Lomb)
3. Some useful extensions: uncertainties (6.1), floating mean (6.2), higher-order modes (6.3)
4. Computing the window function & the CLEAN algorithm
5. FAPS and eFAPS
6. When the LS periodogram fails: ex: P/2 being isolated in rotation curve. 
7. When the LS periodogram fails: correlated noise

Problem ideas:
- compute the actual periodogram and compare it to the LS periodogram
- use astropy periodogram, investigate normalization schemes
- numerically compute window function, eFAP, and LS (incorporating/not 
  uncertainties & floating mean & higher-order modes)
- (more)

"""